In [2]:
import pandas as pd
import mysql.connector
import json

In [3]:
df = pd.read_csv("../data/the_grammy_awards.csv")
print(df.shape)

(4810, 10)


In [4]:
print(df.head())

   year                              title               published_at  \
0  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
1  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
2  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
3  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
4  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   

                  updated_at            category     nominee         artist  \
0  2020-05-19T05:10:28-07:00  Record Of The Year     Bad Guy  Billie Eilish   
1  2020-05-19T05:10:28-07:00  Record Of The Year     Hey, Ma       Bon Iver   
2  2020-05-19T05:10:28-07:00  Record Of The Year     7 rings  Ariana Grande   
3  2020-05-19T05:10:28-07:00  Record Of The Year  Hard Place         H.E.R.   
4  2020-05-19T05:10:28-07:00  Record Of The Year        Talk         Khalid   

                                             workers  \
0  Finneas O'Connell, producer

In [6]:
df.nunique()

year              62
title             62
published_at       4
updated_at        10
category         638
nominee         4131
artist          1658
workers         2366
img             1463
winner             1
dtype: int64

In [5]:
df.isnull().sum()

year               0
title              0
published_at       0
updated_at         0
category           0
nominee            6
artist          1840
workers         2190
img             1367
winner             0
dtype: int64

In [8]:
# replacing nulls
df = df.fillna({'nominee': 'unknown', 'artist': 'unknown', 'workers': 'unknown', 'img': 'unknown'})

In [9]:
df.isnull().sum()

year            0
title           0
published_at    0
updated_at      0
category        0
nominee         0
artist          0
workers         0
img             0
winner          0
dtype: int64

In [10]:
with open('../db_config.json') as config_json:
    config = json.load(config_json)

conx = mysql.connector.connect(**config) 

In [19]:
mycursor = conx.cursor()

mycursor.execute("CREATE TABLE IF NOT EXISTS grammys(year VARCHAR(5), title VARCHAR(40), published_at datetime, updated_at datetime, category VARCHAR(255), nominee VARCHAR(255), artist VARCHAR(255), workers VARCHAR(1000), img VARCHAR(350), winner boolean)")

query = "DESCRIBE grammys"
mycursor.execute(query)

description_table = mycursor.fetchall()

mycursor.close()

desc_table=pd.DataFrame(description_table, columns=['Field', 'Type', 'Null', 'Key', 'Default', 'Extra'])
print(desc_table)

          Field              Type Null Key Default Extra
0          year     b'varchar(5)'  YES  []    None      
1         title    b'varchar(40)'  YES  []    None      
2  published_at       b'datetime'  YES  []    None      
3    updated_at       b'datetime'  YES  []    None      
4      category   b'varchar(255)'  YES  []    None      
5       nominee   b'varchar(255)'  YES  []    None      
6        artist   b'varchar(255)'  YES  []    None      
7       workers  b'varchar(1000)'  YES  []    None      
8           img   b'varchar(350)'  YES  []    None      
9        winner     b'tinyint(1)'  YES  []    None      


In [18]:
#Delete table, just in case :)... delete because of varchar's lengths
# mycursor = conx.cursor()

# query = "DROP table grammys"
# mycursor.execute(query)

# mycursor.close()

True

In [20]:
mycursor = conx.cursor()

# Iterar a través de las filas del DataFrame e insertarlas en la tabla
for index, row in df.iterrows():
    mycursor.execute("""INSERT INTO grammys (year,title,published_at,updated_at,category,nominee,artist,workers,img,winner) 
                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""", (row[0], row[1], row[2], row[3],
                    row[4], row[5], row[6], row[7],
                    row[8], row[9]))


# with open('../data/the_grammy_awards.csv', newline='', encoding="utf8") as csvfile:
#         reader = csv.reader(csvfile, delimiter=',')
#         next(reader) # ignore column names
#         for row in reader:
#             datos= (row[0], row[1], row[2], row[3],
#                     row[4], row[5], row[6], row[7],
#                     row[8], row[9])
#             mycursor.execute(insert, datos)
    
conx.commit()

mycursor.close()

C:\Users\Mrìa Angelica\AppData\Local\Temp\ipykernel_20072\1278858464.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""", (row[0], row[1], row[2], row[3],
C:\Users\Mrìa Angelica\AppData\Local\Temp\ipykernel_20072\1278858464.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[4], row[5], row[6], row[7],
C:\Users\Mrìa Angelica\AppData\Local\Temp\ipykernel_20072\1278858464.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value 

True